# YOLO11 Model Conversion to TensorFlow.js and Upload to Amazon S3
# ----------------------------------------------------------------------------------

In [ ]:
# Step 1: Install Required Libraries
# Install necessary libraries including Ultralytics, TensorFlow, ONNX, ONNX-TF, and TensorFlow.js
!pip install ultralytics tensorflow tensorflowjs onnx onnx-tf

In [ ]:
# Step 2: Load YOLO11 Model and Export to ONNX Format
# Import YOLO from the Ultralytics package, load a YOLO11 model, and export to ONNX format

from ultralytics import YOLO

# Load a pre-trained YOLO11 model; replace "yolo11n.pt" with your desired model file
model = YOLO("yolo11n.pt")  # YOLO11 nano model; you can use other YOLO11 variants, e.g., "yolo11s.pt"

# Export the YOLO11 model to ONNX format
model.export(format="onnx")

# The ONNX file (e.g., "yolo11n.onnx") will be saved in the current working directory.

In [ ]:
# Step 3: Convert ONNX Model to TensorFlow SavedModel
# Use ONNX-TF to load the ONNX model and convert it to TensorFlow SavedModel format

import onnx
from onnx_tf.backend import prepare

# Load the ONNX model generated from the previous step
onnx_model = onnx.load("yolo11n.onnx")

# Convert the ONNX model to TensorFlow SavedModel
tf_rep = prepare(onnx_model)
tf_rep.export_graph("yolo11_saved_model")

# The TensorFlow SavedModel will be stored in the `yolo11_saved_model` directory.

In [ ]:
# Step 4: Convert TensorFlow SavedModel to TensorFlow.js Format
# Use TensorFlow.js converter to transform the SavedModel to a web-compatible format

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model yolo11_saved_model yolo11_web_model

# The `yolo11_web_model` directory will contain a `model.json` and weight files in `.bin` format.

In [ ]:
# Step 5: Upload Converted Model to Amazon S3
# Define the S3 bucket and path where the model files will be uploaded

import boto3
import os

# Initialize the S3 client
s3 = boto3.client('s3')

# Set your bucket name and the target directory within the bucket
bucket_name = 'your-s3-bucket-name'  # Replace with your actual S3 bucket name
upload_prefix = 'models/yolo11_web_model'  # Target S3 directory for the model

# Upload each file in the TensorFlow.js model directory to S3
model_dir = 'yolo11_web_model'
for root, dirs, files in os.walk(model_dir):
    for file in files:
        local_path = os.path.join(root, file)
        s3_key = os.path.join(upload_prefix, file)
        s3.upload_file(local_path, bucket_name, s3_key)
        print(f'Uploaded {local_path} to s3://{bucket_name}/{s3_key}')

# The model files are now available in the specified S3 bucket and directory.

# ----------------------------------------------------------------------------------
# Summary
# - This notebook converts the YOLO11 model from Ultralytics to TensorFlow.js format.
# - The TensorFlow.js model is uploaded to S3, making it accessible for web applications.
# ----------------------------------------------------------------------------------